## **Install Library**

In [ ]:
install.packages("keras")
install.packages("foreach")
install.packages("doSNOW")
install.packages("googledrive")
install.packages("abind")

In [2]:
library(keras)
library(tidyverse)
library(foreach)
library(purrr)
library(doSNOW) # untuk melakukan iterasi secara paralel
library(abind) # untuk bind array

Warning message:
"package 'keras' was built under R version 4.0.3"
-- Attaching packages ------------------------------------------------------------------------------- tidyverse 1.3.0 --

v ggplot2 3.3.2     v purrr   0.3.4
v tibble  3.0.4     v dplyr   1.0.2
v tidyr   1.1.2     v stringr 1.4.0
v readr   1.4.0     v forcats 0.5.0

Warning message:
"package 'tibble' was built under R version 4.0.3"
Warning message:
"package 'tidyr' was built under R version 4.0.3"
Warning message:
"package 'readr' was built under R version 4.0.3"
Warning message:
"package 'dplyr' was built under R version 4.0.3"
-- Conflicts ---------------------------------------------------------------------------------- tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()

Warning message:
"package 'foreach' was built under R version 4.0.3"

Attaching package: 'foreach'


The following objects are masked from 'package:purrr':

    accumulate, when


Warning message:
"

## **Fungsi Untuk Membaca Image dan menjadikan array**

Dengan Pararel iterasi

In [10]:
image_from_directory <- function(alamat, w, h, grayscale = T, ...){
  dir = list.files(alamat, full.names = T)
  allloc = c()
  
  # mendapatkan semua lokasi gambar
  if(str_detect(dir[1], "(png|jpg|jpeg)")) allloc = dir
  else{
    for(i in 1:length(dir)){
      allloc = c(allloc, list.files(dir[i], full.names = T, recursive = T))
    }
  }
  
  # agar tidak over gunakan core - 1
  cores = parallel::detectCores()
  cl <- makeSOCKcluster(cores-1)
  registerDoSNOW(cl)

  # iterasi secara pararel
  hasil <- 
    foreach::foreach(i = 1:length(allloc), .packages = c("keras", "stringr"), ...) %dopar% {
                     label = str_split(allloc[i], pattern = "/")[[1]]
                     label = label[length(label)-1]
                     
                     img = image_load(allloc[i], target_size = c(w, h), grayscale = grayscale)
                     img = image_to_array(img)
                     img = array_reshape(img, c(1, dim(img)))
                     img = img/255
 
                     list(image = img, lab = label)
                   }
  stopCluster(cl)
  # membuat fungsi abind1
  abind1 = function(...) abind::abind(..., along = 1)
  hasil <- list(image = do.call(abind1, map(hasil, "image")),
                label = do.call(c, map(hasil, "lab")))
  
  return(hasil)
}


In [ ]:
image_from_directory <- function(alamat, w, h, grayscale = T){
  dir = list.files(alamat)
  allloc = c(); label = c()
  
  # mendapatkan semua lokasi gambar
  if(str_detect(dir[1], "(png|jpg|jpeg)")) allloc = dir
  else{
    for(i in 1:length(dir)){
      isi = list.files(paste0(alamat,"/",dir[i]), full.names = T, recursive = T)
      allloc = c(allloc, isi)
      label = c(label, rep(dir[i], length(isi)))
    }
  }
  # make progress bar
  pb <- txtProgressBar(1, length(allloc), style = 3)
  
  images = lapply(allloc, readImage) 
  for (i in 1:length(allloc)){
    images[[i]] <- resize(images[[i]], w, h)
    images[[i]] <- toRGB(images[[i]])
    setTxtProgressBar(pb, i)
  }
  close(pb)
  
  # gabung semua image
  cat("combine images")
  images = combine(images)
  # ubah urutan dimensi 
  cat("\nubah urutan dimensi") 
  images = aperm(images, c(4, 1, 2, 3))
  cat("\nDone") 
  return(list(images, label))
}

# **Untuk Google Drive**

In [11]:
library("googledrive")

In [ ]:
# Check if is running in Colab and redefine is_interactive()
if (file.exists("/usr/local/lib/python3.6/dist-packages/google/colab/_ipython.py")) {
  install.packages("R.utils")
  library("R.utils")
  library("httr")
  my_check <- function() {return(TRUE)}
  reassignInPackage("is_interactive", pkgName = "httr", my_check) 
  options(rlang_interactive=TRUE)
}

In [ ]:
# authorize google drive
drive_auth(
  email = gargle::gargle_oauth_email(),
  path = NULL,
  scopes = "https://www.googleapis.com/auth/drive",
  cache = gargle::gargle_oauth_cache(),
  use_oob = gargle::gargle_oob_default(),
  token = NULL
)

In [ ]:
drive_download("Aksara Jawa/train/ba/ba.0.jpg")

# **Import gambar**

Masukkan file zip dulu

In [14]:
# unzip data
unzip("/content/Aksara Jawa.zip")

In [15]:
w = 100
h = 100

path = "/content/Aksara Jawa/train"
train = image_from_directory(path, w, h, grayscale = T)
x_train = train[[1]] 
y_train = train[[2]] 
y_train = factor(y_train, labels = 0:19)

path2 = "/content/Aksara Jawa/test"
test = image_from_directory(path2, w, h, grayscale = T)
x_test = test[[1]] 
y_test = test[[2]] 
y_test = factor(y_test, labels = 0:19)

# **Cek Dimensi**

In [16]:
dim(x_train)
dim(x_test)

# ubah dimensi label
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

dim(y_train)
dim(y_test)

[1] 2520  100  100    1

[1] 620 100 100   1

[1] 2520   20

[1] 620  20

# **Membuat Model**

In [3]:
model <- keras_model_sequential() %>% 
  layer_conv_2d(filters = 32, kernel_size = 3,
                padding = "same", input_shape = c(w, h, 1), activation = "relu") %>% 
  layer_conv_2d(filters = 32, kernel_size = 3,
                padding = "same", activation = "relu") %>% 
  layer_max_pooling_2d(pool_size = 2) %>% 
  
  layer_conv_2d(filters = 64, kernel_size = 3,
                padding = "same", activation = "relu") %>% 
  layer_conv_2d(filters = 64, kernel_size = 3,
                padding = "same", activation = "relu") %>% 
  layer_max_pooling_2d(pool_size = 2) %>% 
  
  layer_conv_2d(filters = 128, kernel_size = 3,
                padding = "same", activation = "relu") %>% 
  layer_conv_2d(filters = 128, kernel_size = 3,
                padding = "same", activation = "relu") %>% 
  layer_max_pooling_2d(pool_size = 2) %>%

  layer_flatten() %>%
  layer_dense(32, activation = "relu") %>%
  layer_dropout(0.5) %>% 
  
  layer_dense(20, activation = "softmax")


summary(model)

ERROR: Error in normalize_shape(input_shape): object 'w' not found


# **Compile**

In [130]:
# compile
model %>% compile(
  loss = "categorical_crossentropy",
  optimizer = optimizer_rmsprop(),
  metrics = "accuracy"
)

# **Fit**

In [131]:
history <- model %>%
  fit(x_train,
      y_train,
      batch_size = 30,
      epoch = 70,
      verbose = 1
  )

In [132]:
history
# plot(history)

model %>% evaluate(x_test, y_test) 


Final epoch (plot to see history):
    loss: 0.09934
accuracy: 0.9639 

loss  accuracy 
0.8520818 0.8612903

Save Model

In [108]:
save_model_hdf5(model, "model_aksara_jawa100_4.h5")

In [111]:
model <- load_model_hdf5("/content/model_aksara_jawa100_4.h5")

# **Confusion Matrix**

In [112]:
hasil = model %>% predict_classes(x_test)
actual = test[[2]] 

table(prediksi = hasil, actual)

        actual
prediksi ba ca da dha ga ha ja ka la ma na nga nya pa ra sa ta tha wa ya
      0  28  0  0   0  0  0  0  0  0  0  0   2   2  0  0  0  0   3  2  0
      1   0 26  0   0  0  0  0  0  0  0  0   0   0  0  0  2  0   0  0  0
      2   0  0 31   0  0  0  0  0  0  0  0   0   0  0  0  0  0   0  0  0
      3   0  0  0  28  0  0  0  0  0  0  0   0   0  4  0  0  0   0  1  0
      4   0  0  0   0 28  0  0  0  0  0  0   0   0  0  0  0  0   0  0  0
      5   2  0  0   0  0 29  0  0  0  0  0   0   0  0  0  0  6   0  0  0
      6   0  0  0   0  0  0 31  0  0  0  0   0   0  0  0  0  0   0  0  0
      7   0  0  0   2  0  1  0 31  0  0  0   1   0  0  0  0  1   0  0  0
      8   0  0  0   0  0  0  0  0 31  0  0   0   0  1  0  0  0   0  0  1
      9   0  0  0   0  0  0  0  0  0 31  0   0   0  0  0  0  0   0  0  0
      10  0  0  0   0  0  0  0  0  0  0 31   0   0  0  0  0  0   0  0  0
      11  0  0  0   0  0  0  0  0  0  0  0  28   0  0  0  0  0   7  3  0
      12  1  0  0   0  0  0  0  0  0

In [ ]:
# Dengan ggplot2
table(prediksi = hasil, actual) %>% 
  as.data.frame() %>% 
  ggplot()+
  geom_tile(aes(x = actual, y = prediksi, fill = Freq))+
  geom_text(aes(x = actual, y = prediksi, label = Freq), col = "white")+
  theme(legend.position = "none")

# **Prediksi Beberapa Gambar**

In [152]:
aksara = c("ba" ,"ca" ,"da" ,"dha","ga" ,"ha" ,"ja" ,"ka" ,
           "la" ,"ma" ,"na" ,"nga", "nya" ,"pa" ,"ra" ,"sa" ,
           "ta" ,"tha" ,"wa" ,"ya")

In [157]:
# ambil nama file gambar
img = list.files("/content/Aksara Jawa/predict")

# ambil array gambar
gambar = image_from_directory("/content/Aksara Jawa/predict", w, h, grayscale = T)
gambar = gambar[[1]]
dim(gambar)

# cara subset array
dim(gambar[1:3, , , ,drop = F])

[1] "na1.png"

[1]  25 100 100   1

[1]   3 100 100   1

In [165]:
indeks = 1

# aslinya
print("Nama Gambar")
img[indeks]


#output predict clasess 0-19
print("prediksi")
model %>% predict_classes(gambar[indeks, , , ,drop = F]) %>% +1 %>% aksara[.]

[1] "Nama Gambar"


[1] "ba17.png"

[1] "prediksi"


[1] "wa"